In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
def contraction(params,x,p):
    #beta and x are kind of parameters. x is the empirical distribution of x?
    k = int(x.shape[1]/2)
    util1 = np.dot(x[:,0:k], params[0:k])  + params[k]*p[0]
    util2 = np.dot(x[:,k:], params[k+1:2*k+1]) + params[2*k+1]*p[1]
    contr_result = [np.exp(util1)/(1+np.exp(util1)),np.exp(util2)/(1+np.exp(util2))]
    return np.array(contr_result)

#actually caclualte an equilibrium in this game
N=1000
ps = np.array([.5,.5])
betas = np.array([1,-2, 2,-1])

#set up xs
xs = np.random.normal(scale=2, size=(N,2)) #assumes xs are independent? could use a copula?
xs = pd.DataFrame(xs,columns = ['x11','x12'])
xs = np.array(xs)

print(contraction(betas,xs,ps))

[[0.31853016 0.67224594 0.05784818 ... 0.53149543 0.01986203 0.07804578]
 [0.79185404 0.99999001 0.0024764  ... 0.90815367 0.0340166  0.86508021]]


In [3]:
def contraction_map(betas,x,p):
    """final result is beliefs of firm1/firm2"""
    for i in range(50):
        #print('b1: %.4f, p2: %.4f'%tuple(p))
        p = contraction(betas,x,p).mean(axis=1)
        p = np.flip(p)
        #print('p1: %.4f, p2: %.4f'%tuple(p))
        #print('----- end of iteration %s ----'%i)
    return p

result_ps = contraction_map(betas,xs,ps)
us = np.random.logistic(size=(1000,2))

k = int(xs.shape[1]/2)
es = np.random.logistic(0, 1, (2,N) )
y1 = 1*(np.dot(xs[:,0:k], betas[0:k])  + betas[k]*result_ps[0] + es[0,:] >= 0)
y2 = 1*(np.dot(xs[:,k:], betas[k+1:2*k+1]) + betas[2*k+1]*result_ps[1] + es[1,:] >= 0)
ys = np.array([y1,y2]).transpose()
print(np.flip(result_ps))
print(ys.mean(axis=0))

[0.36783766 0.47134179]
[0.352 0.449]


In [4]:
result_df = np.concatenate( (ys,xs ) ,axis=1)
result_df = pd.DataFrame(result_df, columns=['y1','y2','x11','x12'])
print(result_df)
result_df.to_csv('monte_carlo.csv')

      y1   y2       x11       x12
0    0.0  1.0  0.239465  0.918069
1    0.0  1.0  1.718361  6.006896
2    0.0  0.0 -1.790344 -2.749234
3    1.0  1.0  2.403576  0.774506
4    1.0  0.0  2.137133 -2.482338
..   ...  ...       ...       ...
995  0.0  1.0 -0.063944  1.311125
996  1.0  0.0 -2.163377 -1.125970
997  1.0  1.0  1.126149  1.395648
998  0.0  0.0 -2.898883 -1.423149
999  0.0  1.0 -1.469200  1.179071

[1000 rows x 4 columns]


In [8]:
class BayesNashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        
        p = self.endog.mean(axis=0)
        p = contraction_map(params,self.exog,p)
        
        likelihood = contraction(params,self.exog,p).transpose()
        ll = self.endog*np.log(likelihood) + (1-self.endog)*np.log(1-likelihood)
        
        return -1*ll.sum()
        
        
    
    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        start_params = np.ones(self.exog.shape[1]+2)
        return super(BayesNashLogit, self).fit(start_params=start_params,**kwds)

        
        
N = result_df.shape[0]
ys = result_df[['y1','y2']]
xs = result_df[['x11' ,'x12']]


model = BayesNashLogit(ys,xs)
model_fit = model.fit(xtol=1e-12,ftol=1e-12)
print(model_fit.summary(xname=["b11","d1","b12","d2"]))

Optimization terminated successfully.
         Current function value: 0.713641
         Iterations: 234
         Function evaluations: 475
                            BayesNashLogit Results                            
Dep. Variable:           ['y1', 'y2']   Log-Likelihood:                -713.64
Model:                 BayesNashLogit   AIC:                             1431.
Method:            Maximum Likelihood   BIC:                             1441.
Date:                Wed, 07 Apr 2021                                         
Time:                        15:34:33                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

In [9]:
#attempt at standard errors...
H = np.linalg.inv(model.hessian(model_fit.params)/N)
diag = np.diagonal(np.linalg.inv(model.hessian(model_fit.params)))
print(np.sqrt(-1*diag))

[0.06419419 0.21774813 0.12739173 0.32234286]
